In [ ]:
# Import 
import os
import time
import datetime

from pathlib import Path

from invagination.ellipsoid import initiate_ellipsoid
from invagination.delamination import delamination_process

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
SIM_DIR = Path('/home/admin-suz/Documents/Simulations/')
today = datetime.date.today()

sim_save_dir = SIM_DIR/f'{today.isoformat()}_YolkElasticityTest'

try:
    os.mkdir(sim_save_dir)
except FileExistsError:
    pass

In [ ]:
class RadialTension(effectors.AbstractEffector):
    
    dimensions = units.line_tension
    magnitude = 'radial_tension'
    label = 'Apical basal tension'
    element = 'vert'
    specs = {'vert':{'is_active',
             'height',
             'radial_tension'}}

    @staticmethod
    def energy(eptm):
        return eptm.vert_df.eval(
            'height * radial_tension * is_active')
         
    @staticmethod
    def gradient(eptm):
        grad = height_grad(eptm) * to_nd(
            eptm.vert_df.eval('radial_tension'), 3)
        grad.columns = ['g'+c for c in eptm.coords]
        return grad, None
    
    



EllipsoidBModel = model_factory(
    [
    RadialTension,
    VitellineElasticity,
    effectors.FaceContractility,
    effectors.FaceAreaElasticity,
    effectors.CellVolumeElasticity,
    ], effectors.FaceAreaElasticity)


model = EllipsoidBModel
geom = EllipsoidBGeometry

In [ ]:
def init():
    dsets = hdf5.load_datasets('../data/hf5/ellipsoid_sheet_init.hf5',
                               data_names=['vert', 'edge', 'face', 'cell'])

    with open('../data/json/ellipsoid.json', 'r+') as fp:
        specs = json.load(fp)

    sheet = Sheet('ellipse', dsets, specs)

    # Modify some initial value
    sheet.settings['threshold_length'] = 1e-3
    sheet.settings['vitelline_space'] = 0.2
    sheet.vert_df['radial_tension'] = 0.
    sheet.cell_df['prefered_vol'] = 4539601.384437251
    
    return sheet


In [ ]:
solver_kw = {'minimize': {'method': 'L-BFGS-B',
                          'options': {'ftol': 1e-8,
                                      'gtol': 1e-8}}}

## Test several value of yolk volume elasticity

In [ ]:

volume = [1.e-11, 1.e-10, 1.e-9, 5.e-9, 1.e-8, 2.e-8, 3.e-8,  4.e-8, 7.e-8, 8.e-8,
          1.e-7,2.e-7, 3.e-7, 4.e-7, 7.e-7, 1.e-6, 3.e-6, 1.e-5, 1.e-4, 1.e-3, 1.e-2, 1.e-1]

for v in volume : 
    sheet=init()
    print (v)
    sheet.cell_df['vol_elasticity'] = v
    
    solver = Solver
    res = solver.find_energy_min(sheet, geom, model, **solver_kw)
    
    tp = (v, sheet.cell_df.vol/sheet.cell_df.prefered_vol)
    volume_dataframe.loc[v, 'elasticity'] = tp[1][0]
    volume_dataframe.to_csv('resultats_yolk.csv')

In [ ]:
volume_dataframe = pd.DataFrame.from_csv('resultats_yolk.csv')
fig = plt.figure()
ax = fig.add_subplot(111)
volume_dataframe = volume_dataframe.drop(volume_dataframe[volume_dataframe.elasticity == 0].index)
ax.plot(volume_dataframe.index, (1-volume_dataframe.elasticity)*100, '.', color = 'black')

ax.plot(3.e-6, (1-volume_dataframe.loc[3.e-6, 'elasticity'])*100, 'o',color = 'red')
ax.axvline(x=3.e-6,ymin=0, ymax=0.08, linewidth=1,linestyle='--', color='red')
ax.axhline(y=(1-volume_dataframe.loc[3.e-6, 'elasticity'])*100, xmin=0, xmax=0.55,
           linewidth=1,linestyle='--', color='red')


ax.set_xscale("log")
ax.set_xlabel('Yolk elasticity', size=12, family='serif')
ax.set_ylabel('Compression (%)', size=12, family='serif')


plt.savefig('../paper/yolkElasticity.svg', dpi = 300)

